YOLOv10 라이브러리 링크: https://docs.ultralytics.com/ko/models/yolov10/#what-is-yolov10-and-how-does-it-differ-from-previous-yolo-versions

YOLOv10
- Real-Time End-to-End 객체 탐지 방식 도입
- NMS(비최대 억제) 제거
- 이중 레이블 할당(diagnose dual assignments)방식 적용
- 이전 버전의 YOLO 모델 대비 동일 성능을 내는 YOLOv10의 모델들은 더 빠르고 매개변수와 연산량이 더 적음

기존 YOLO 모델의 한계
- 기존 YOLO의 객체 감지 파이프라인은 모델 포워드 프로세스와 NMS 후처리로 구성되며, 이에 결함이 있어 최적이 아닌 정확도와 지연을 초래함
 - 기존 모델 아키텍처 설계는 여러가지 기법들을 활용했지만 종합적인 검토가 부족하여 비효율적인 계산 중복이 존재함
 - NMS는 1:N 전략을 사용하여 실시간 달성에 방해되는 추론 속도 저하 문제가 있음

Consistent Dual Assignments for NMS-free Training
- 기존 YOLO 모델은 한 객체에 여러 예측을 할당하는 방식을 사용 -> supervisory signal 제공, 후처리 과정에서 NMS가 필요함
- 이중 레이블 할당 및 일관된 매칭 메트릭(consistent matching metric)을 사용
- 학습시 1:N 헤드와 1:1 헤드를 모두 사용하여 최적화 -> 학습 효과 증대
- 추론시 1:1 헤드만 사용 -> NMS 후처리 작업 x

Holisitic Efficiency-Accuracy Driven Model Design
- 기존 YOLO는 다운샘플링과 채널 증가변환을 동시에하는데 이를 분리하는 것을 제안
- 모델의 효율성과 정확성을 최적화하기 위해 경량화된 분류 헤드(lightweight classification head), 공간-채널 분리 다운샘플링(spatial-channel decoupled downsapling), 순위 기반 블록 설계(rank-guided block design) 도입
- 연산 중복을 줄여 모델의 효율성을 높이고, 대규모 커널 컨볼루션(large-kernel convolution) 및 partial self-attention module을 통해 적은 비용으로 성능을 개선
- 기존 YOLO의 구조를 intrinsic rank에 따라 중복성을 검사
- 벡본과 넥이 단계에 따라 8단계까지 구분되며 스테이지의 마지막 컨볼루션에서 임계값보다 큰 값의 수를 집계
- 규모가 큰 모델일 수록 값이 작은 스테이지가 많으며 이 모델이 더 많은 중복성을 내포 -> 중복성을 해결하기 위해 rank-guided block design을 제시하고, 이에 들어가는 모듈로 CIB(compact inverted block)을 제안
- 중복성의 오름차순에 따라 해당 스테이지의 기본 블록을 CIB로 교체하고 성능 변화를 추가로 검사
- 성능 변화가 없으면 단계별로 교체

Enhanced Model Capabilities
- 대규모 커널 컨볼루션과 부분적 셀프 어텐션 모듈을 통하바여 큰 계산 비용 없이 성능을 개선

Model Architecture
1. Backbone
 - 특징 추출을 담당
 - 향상된 버전의 CSPNet을 사용하여 그라데이션 흐름을 개선하고 계산 중복성을 줄임

2. Neck
 - 다양한 스케일의 피처를 취합하여 헤드로 전달함
 - 효과적인 멀티스케일 피처 융합을 위한 경로 집계 네트워크(PAN) 레이어가 포함됨

3. One-to-Many Head
 - 훈련 중 객체당 여러 예측을 생성하여 supervisory signal 제공 및 학습 정확도 향상

4. One-to-One Head
 - 추론 중 객체당 하나의 최적 예측을 생성
 - NMS를 제거하여 대기 시간을 줄이고 효율성 개선

In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv10n model
model = YOLO("yolov10n.pt")

# Perform object detection on an image
results = model("image.jpg")

# Display the results
results[0].show()